In [25]:
import torch
import torchvision
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, Dropout, Input
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

In [26]:
objects = [
    'ambulance',
    'army vehicle',
    'auto rickshaw',
    'bicycle',
    'bus',
    'car',
    'garbagevan',
    'human hauler',
    'minibus',
    'minivan',
    'motorbike',
    'pickup',
    'policecar',
    'rickshaw',
    'scooter',
    'suv',
    'taxi',
    'three wheelers -CNG-',
    'truck',
    'van',
    'wheelbarrow'
]

In [27]:
def load_data(data_dir, upto=np.inf):
    
    images = []
    annotations = []
    i = 0

    images_dir = os.path.join(data_dir, 'train', 'images')
    labels_dir = os.path.join(data_dir, 'train', 'labels')

    for filename in os.listdir(images_dir):
        if i == upto:
            break
        if filename.endswith('.jpg'):
            img_path = os.path.join(images_dir, filename)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (224, 224))
            images.append(img)

            label_path = os.path.join(labels_dir, filename.replace('.jpg', '.txt'))
            with open(label_path, 'r') as f:
                lines = f.readlines()
                boxes = []
                classes = []
                for line in lines:
                    values = line.strip().split()
                    class_id = int(values[0])
                    x, y, w, h = map(float, values[1:])
                    boxes.append([x, y, w, h])
                    classes.append(class_id)
                annotations.append({'boxes': boxes, 'classes': classes})
        
        i += 1
        print(f'For iter {i} collected image: {filename}')

    return np.array(images), annotations

def object_outline(image, annotation, names=False):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    
    ax.imshow(image)
    
    for i, box in enumerate(annotation['boxes']):
        x_center, y_center, width, height = box
        
        x_min = int((x_center - width / 2) * image.shape[1])
        y_min = int((y_center - height / 2) * image.shape[0])
        x_max = int((x_center + width / 2) * image.shape[1])
        y_max = int((y_center + height / 2) * image.shape[0])
        
        rect = patches.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, linewidth=1, edgecolor='r', facecolor='none')
        if names:
            ax.text(x_min, y_min, objects[annotation['classes'][i]], fontsize=12, color='r')
        ax.add_patch(rect)
    
    plt.show()

def process_prepare(annotations):
    max_boxes = max(len(ann['boxes']) for ann in annotations)
    max_classes = max(len(ann['classes']) for ann in annotations)
    
    for ann in annotations:
        ann['boxes'] += [[-1.0, -1.0, -1.0, -1.0]] * (max_boxes - len(ann['boxes']))
        ann['classes'] += [-1] * (max_classes - len(ann['classes']))
    
    boxes_array = np.array([ann['boxes'] for ann in annotations])
    classes_array = np.array([ann['classes'] for ann in annotations])
    
    return boxes_array, classes_array


In [33]:
images, annotations = load_data('trafic_data', 100)

For iter 1 collected image: 01_jpg.rf.8d8a2f0f90d5b83893cd252acd832c93.jpg
For iter 2 collected image: 02_jpg.rf.65a084066fc353cd023eb5c953f40efe.jpg
For iter 3 collected image: 03_jpg.rf.4532f2db68433995da09f54e215160f3.jpg
For iter 4 collected image: 04_jpg.rf.c0b83432c4d09c7cbeaac18e14c4a54a.jpg
For iter 5 collected image: 05_jpg.rf.11a2e7ce391a1e74960099f7923f27f5.jpg
For iter 6 collected image: 06_jpeg.rf.14bdc2005d8029bbe9f24fad4b6af7ea.jpg
For iter 7 collected image: 07_jpg.rf.8447b11632c1b63ab0e127f16625e0d2.jpg
For iter 8 collected image: 08_jpg.rf.837e69c4e93e62472911c1d18be143a4.jpg
For iter 9 collected image: 09_jpg.rf.42406b1c067f04bf73349bd75b2e3fa8.jpg
For iter 10 collected image: 100_jpg.rf.971dbb99e07828d1655d69bbfb7136be.jpg
For iter 11 collected image: 101_jpg.rf.2737e79396d350c7eeb5bf867c346077.jpg
For iter 12 collected image: 102_jpg.rf.2d884248dcc60b7d6b93f61df77921df.jpg
For iter 13 collected image: 103_jpg.rf.cb820e037e0ccc6163377e9ead1fea10.jpg
For iter 14 coll

In [34]:
boxes, classes = process_prepare(annotations)